In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,58793
2,Huelva,Confirmados PDIA 14 días,6582
3,Huelva,Tasa PDIA 14 días,"1282,6158972660132"
4,Huelva,Confirmados PDIA 7 días,2711
5,Huelva,Tasa PDIA 7 dias,"528,2849737903618"
6,Huelva,Total Confirmados,59006
7,Huelva,Curados,47765
8,Huelva,Fallecidos,427


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  58793.0


/tmp/ipykernel_3307/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  16828.0


/tmp/ipykernel_3307/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_3307/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3307/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3307/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 166 personas en los últimos 7 días 

Un positivo PCR cada 71 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,58793.0,6582.0,2711.0,513170.0,528.284974,1282.615897,1065.0
Huelva-Costa,34504.0,3635.0,1462.0,289548.0,504.924917,1255.404976,519.0
Huelva (capital),16828.0,2033.0,867.0,143837.0,602.765631,1413.405452,297.0
Sierra de Huelva-Andévalo Central,6214.0,1275.0,530.0,67391.0,786.455165,1891.944028,275.0
Condado-Campiña,17629.0,1663.0,714.0,156231.0,457.015573,1064.449437,272.0
Valverde del Camino,1402.0,376.0,140.0,12750.0,1098.039216,2949.019608,73.0
Cartaya,2605.0,204.0,94.0,20083.0,468.057561,1015.784494,44.0
Lepe,3454.0,231.0,104.0,27880.0,373.027260,828.550933,41.0
Almonte,2745.0,194.0,85.0,24507.0,346.839678,791.610560,41.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,79.0,32.0,6.0,614.0,977.198697,5211.726384,4.0
Campofrío,88.0,25.0,11.0,713.0,1542.776999,3506.311360,4.0
Galaroza,110.0,44.0,9.0,1382.0,651.230101,3183.791606,4.0
Valverde del Camino,1402.0,376.0,140.0,12750.0,1098.039216,2949.019608,73.0
Rosal de la Frontera,130.0,47.0,29.0,1697.0,1708.898055,2769.593400,19.0
Jabugo,147.0,55.0,27.0,2260.0,1194.690265,2433.628319,9.0
Hinojales,10.0,8.0,8.0,330.0,2424.242424,2424.242424,NaN
Aroche,215.0,72.0,32.0,3054.0,1047.806156,2357.563851,17.0
Fuenteheridos,49.0,15.0,4.0,709.0,564.174894,2115.655853,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Granado (El),55.0,6.0,1.0,532.0,187.969925,1127.819549,0.0,0.166667
San Silvestre de Guzmán,79.0,32.0,6.0,614.0,977.198697,5211.726384,4.0,0.187500
Sanlúcar de Guadiana,43.0,5.0,1.0,403.0,248.138958,1240.694789,1.0,0.200000
Puerto Moral,26.0,5.0,1.0,278.0,359.712230,1798.561151,0.0,0.200000
Galaroza,110.0,44.0,9.0,1382.0,651.230101,3183.791606,4.0,0.204545
Villanueva de los Castillejos,266.0,40.0,10.0,2825.0,353.982301,1415.929204,1.0,0.250000
Fuenteheridos,49.0,15.0,4.0,709.0,564.174894,2115.655853,4.0,0.266667
Puebla de Guzmán,234.0,45.0,12.0,3092.0,388.098318,1455.368693,6.0,0.266667
Zalamea la Real,183.0,40.0,11.0,3054.0,360.183366,1309.757695,7.0,0.275000
